In [2]:
from math import floor

class Parking_Garage():
    def __init__(self):
        self.tickets = ["1","2","3","4","5","6","7","8","9","10"]
        self.spaces = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        self.current_ticket ={}
        self.s = None
    

    def take_ticket(self):
        while True:
            if len(self.spaces) > 0:
                t = self.tickets.pop()
                self.s = self.spaces.pop()

                self.current_ticket[t] = 'unpaid'
                print(f"Your ticket number is {t}. Please keep your ticket when you go to pay.")
                break
        
            else:
                print("There are no move available spaces! Please try again later")
                break

    def leave_garage(self):
        ticket_number = input("\nWhat is your ticket number? ")
        if ticket_number in self.current_ticket.keys():
            print("Ticket Verified")
            if self.current_ticket[ticket_number] == 'unpaid':
                print("You haven't paid your ticket. Please pay before you leave the garage.")
                self.pay_ticket(ticket_number)
            if self.current_ticket[ticket_number] == 'paid':
                self.tickets.append(ticket_number)
                self.spaces.append(int(ticket_number))
                del self.current_ticket[ticket_number]
                del self.current_ticket[int(ticket_number)]
                print("Thank you. Have a nice day.")
                
        else:
            response = input("\nThat is not a valid ticket number...Do you have a ticket? y/n.")
            if response.lower().strip() == 'y':
                self.leave_garage()
            else:
                print("Please take a ticket first.")

    def pay_ticket(self, ticket_number = None):
        if ticket_number == None:
                ticket_number = input("\nWhat is your ticket number? ")
        if ticket_number.strip() in self.current_ticket.keys():
            print("Ticket Verified")
            cost = '$10'
            while True:
                paid = input(f"\nPlease enter {cost} to pay for ticket. ")
                if paid.strip() == cost:
                    self.current_ticket[ticket_number] = 'paid'
                    print("Your ticket has been paid please leave in 15 mins")
                    break

                else:
                    print(f"Invalid attempt, please enter {cost} to pay for ticket.")
                    continue
        else:
            response = input("\nThat is not a valid ticket number...Do you have a ticket? y/n.")
            if response.lower().strip() == 'y':
                self.pay_ticket()
            else:
                print("Please take a ticket first.")
                

marvel_parking = Parking_Garage()        

def run():
    print("\nWelcome to Marvel parking garage! ")

    home = "\nSelect one: Take Ticket / Pay Ticket / Leave Garage / Quit: "
    while True:
        response = input(home)
        if response.lower().strip() == 'take ticket':
            marvel_parking.take_ticket()
        elif response.lower().strip() == 'pay ticket':
            marvel_parking.pay_ticket()
        elif response.lower().strip() == 'leave garage':
            marvel_parking.leave_garage()
        elif response.lower().strip() == 'quit':
            print("Have a good day!")
            break
        else:
            print("Invalid Response...Try Again.")  
run()


Welcome to Marvel parking garage! 

Select one: Take Ticket / Pay Ticket / Leave Garage / Quit: take ticket
Your ticket number is 10. Please keep your ticket when you go to pay.

Select one: Take Ticket / Pay Ticket / Leave Garage / Quit: quit
Have a good day!


In [3]:
from flask import Blueprint, request, jsonify
from car_inventory.helpers import token_required
from car_inventory.models import User, Car, car_schema, cars_schema, db

api = Blueprint('api', __name__, url_prefix='/api')

@api.route('/getdata')
def getdata():
    return { 'some': 'value'}


# CREATE CAR ENDPOINT
@api.route('/cars', methods = ['POST'])
@token_required
def create_car(current_user_token):
    make = request.json['make']
    model = request.json['model']
    sale_price = request.json['sale_price']
    color = request.json['color']
    year = request.json['year']
    mpg = request.json['mpg']
    new_used = request.json['new_used']
    user_token = current_user_token.token

    car = Car(make,model,sale_price,color,year,mpg,new_used,user_token = user_token)

    db.session.add(car)
    db.session.commit()


    response = car_schema.dump(car)
    return jsonify(response) 


# RETRIEVE ALL CARS ENDPOINT
@api.route('/cars', methods = ['GET'])
@token_required
def get_cars(current_user_token):
    # set owner equal to 
    owner = current_user_token.token
    # .all to get everthing
    cars = Car.query.filter_by(user_token = owner).all()
    response = cars_schema.dump(cars)
    return jsonify(response)


# RETRIEVE ONE CAR BY ID
@api.route('/cars/<id>', methods = ['GET'])
@token_required
def get_car(current_user_token, id):
    car = Car.query.get(id)
    response = car_schema.dump(car)
    return jsonify(response)


# UPDATE CAR ENDPOINT
@api.route('/cars/<id>', methods = ['POST', 'PUT'])
@token_required
def update_car(current_user_token, id):
    # Grabbing the car from the table - instance is denoted by the id
    car = Car.query.get(id)  #Getting a car instance

    # Then grab each individual attribute and update zero or more of the following values
    car.make = request.json['make']
    car.model = request.json['model']
    car.sale_price = request.json['sale_price']
    car.color = request.json['color']
    car.year = request.json['year']
    car.mpg = request.json['mpg']
    car.new_used = request.json['new_used']
    car.user_token = current_user_token.token

    # Then commit it to the database
    db.session.commit()
    response = car_schema.dump(car)
    return jsonify(response)


# DELETE CAR ENDPOINT
@api.route('/cars/<id>', methods = ['DELETE'])
@token_required
def delete_car(current_user_token, id):
    car = Car.query.get(id)
    db.session.delete(car)
    db.session.commit()
    response = car_schema.dump(car)
    return jsonify(response)

ModuleNotFoundError: No module named 'car_inventory'